# Aprendizado de conluio em licitações públicas
## Etapas do pré-processamento

In [1]:
#!pip install pyodbc
#!pip install mlxtend
#!pip install xlrd
#!pip install lime
#!pip install shap
#!pip install imblearn
#!pip install seaborn
#!pip install psycopg2

In [2]:
import pandas as pd
import gc

## Carregar dados de licitações do governo Federal

#### Fonte: https://portaldatransparencia.gov.br/download-de-dados/licitacoes

### Carregar dados - Licitações

In [3]:
import os

In [ ]:
anos = ['2013', '2014', '2015', '2016', '2017', '2018']

In [ ]:
def get_files_dir (pasta, ano):
    df = pd.DataFrame()
    for a in os.listdir(pasta):
        arquivo = pasta+'\\'+a
        if os.path.isfile(arquivo):     
            df_t = pd.read_csv(arquivo, sep=';', encoding='utf-8', low_memory=False)
            df = pd.concat([df, df_t], ignore_index=True)
        elif a == ano:
            df_t = get_files_dir (arquivo, ano)
            df = pd.concat([df, df_t], ignore_index=True)
    return df 

In [ ]:
def get_files (dir, ano):   
    df = get_files_dir (dir, ano)
    df = df.drop_duplicates(ignore_index=True)
    #Só faz sentido avaliar quem tem concorrente
    df = df.loc[(df['Modalidade Compra'] != "Inexigibilidade de Licitação") & (df['Modalidade Compra'] != "Dispensa de Licitação")]
    return df

In [ ]:
#Carregar a partir dos arquivos
dir_licit = 'C:\\Users\\lucas\\Downloads\\Cefet-RJ\\Licitacao'
dir_item = 'C:\\Users\\lucas\\Downloads\\Cefet-RJ\\ItemLicitacao'
dir_partic = 'C:\\Users\\lucas\\Downloads\\Cefet-RJ\\ParticipantesLicitacao'
df_u = pd.DataFrame()
for a in anos:
    print ('Ano: '+a)
    df_item = get_files (dir_item, a)
    print ('Total Item: ', df_item.shape)
    df_partic = get_files (dir_partic, a)
    print ('Total Participantes: ',df_partic.shape)
    df_t = df_partic.merge(df_item, left_index=False, right_index=False, how="inner")
    df_licit = get_files (dir_licit, a)
    print ('Total Licitações: ',df_licit.shape)
    df_t = df_t.merge(df_licit, left_index=False, right_index=False, how="inner")
    print ('Total: ',df_t.shape)
    del(df_item, df_partic, df_licit)
    n = gc.collect()
    #Antecipa exclusão de colunas código (redundantes) para evitar problemas de memória
    cols = [x for x in df_t.columns if 'Código' in x]
    df_t.drop(columns=cols, inplace=True)
    df_u = pd.concat([df_u, df_t], ignore_index=True)
    del(df_t)
print ('Total')
print (df_u.shape)

In [ ]:
df_u.head(50)

In [ ]:
n = gc.collect()
print("Número de objetos coletados pela função GC:", n)
print("Objetos restantes:", gc.garbage)

### Carregar dados - Conluios

In [ ]:
df_cc = pd.read_excel('G:\\Meu Drive\\Cefet-rj\\Projeto de pesquisa\\mapeamento-conluios.xlsx', index_col=None, sheet_name='conluios-participantes') 

In [ ]:
df_cc.head()

In [ ]:
#Padroniza nomes colunas
df_cc.columns = df_cc.columns.str.upper().str.replace(' ', '_')
df_cc.columns = df_cc.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [ ]:
#Padroniza valores
df_cc = df_cc.apply(lambda x: x.astype(str).str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))

In [ ]:
df_cc = df_cc.dropna()

## Análise exploratória dos dados e Pré-processamento

In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
def escalar_df(df_t):
    df = df_t.copy()
    objList = df.select_dtypes(include = ['object', 'category']).columns
    labelencoder = LabelEncoder()
    for feat in objList:
        df[feat] = labelencoder.fit_transform(df[feat].astype(str))
    scaler = MinMaxScaler()
    scaler.fit(df)
    scaled = scaler.fit_transform(df)
    scaled_df = pd.DataFrame(scaled, columns=df.columns)
    return scaled_df

### Informações gerais dos atributos

In [ ]:
#Avalia atributos com muitos valores únicos
df = df_u.applymap(str)
df.describe(include=[object])

In [ ]:
#Avalia atributos com muitos valores nulos
print(df_u.isnull().sum())

In [ ]:
#Avalia atributos com baixa variância (contribuem pouco para o aprendizado)
df_u_scaled = escalar_df(df_u)
print(df_u_scaled.var())

In [ ]:
#Avalia tipo dos atributos
print(df_u.dtypes)

### Seleção de dados

In [ ]:
#Padroniza nomes colunas
df_u.columns = df_u.columns.str.upper().str.replace(' ', '_')
df_u.columns = df_u.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [ ]:
df_u['QTD_PARTICIPANTES_ITEM'] = df_u.groupby(['NUMERO_LICITACAO', 'NOME_UG', 'DESCRICAO_ITEM_COMPRA'])['NOME_PARTICIPANTE'].transform('nunique')

In [ ]:
#Remove casos de licitações com apenas um participante
df_u = df_u.loc[df_u['QTD_PARTICIPANTES_ITEM'] > 1]

In [ ]:
n = gc.collect()
print("Número de objetos coletados pela função GC:", n)
print("Objetos restantes:", gc.garbage)

In [ ]:
#Remove as linhas que o participante é o vencedor
df_u = df_u.loc[df_u['FLAG_VENCEDOR'] != 'SIM']

In [ ]:
print (df_u.shape)
print (df_u.columns)

### Seleção de atributos, transformações e limpezas

In [ ]:
#Descrição = Descrição Item
del df_u['DESCRICAO']
#Muitos valores vazios
del df_u['DATA_ABERTURA']
#Muitos valores únicos
del df_u['OBJETO']
del df_u['NUMERO_PROCESSO']
#Nome Órgão Superior tem baixa variabilidade, além disso, mais importante a identificação mais granular (Nome UG)
del df_u['NOME_ORGAO_SUPERIOR']
del df_u['NOME_ORGAO']
#Nome do vencedor já é um campo, pra que indicar se ele venceu?
del df_u['FLAG_VENCEDOR']
#Mais relevante será considerar o valor individual do item
del df_u['VALOR_LICITACAO']

In [ ]:
df_u['RESULTADO_COMPRA_MES'] = df_u['DATA_RESULTADO_COMPRA'].apply(lambda x : x[3:5])
df_u['RESULTADO_COMPRA_ANO'] = df_u['DATA_RESULTADO_COMPRA'].apply(lambda x : x[6:10])
del df_u['DATA_RESULTADO_COMPRA']

In [ ]:
df_u['MUNICIPIO'] = df_u['MUNICIPIO'] + "-" + df_u['UF']
del df_u['MUNICIPIO']

In [ ]:
df_u['VALOR_ITEM'] = df_u['VALOR_ITEM'].apply(lambda x: x.replace(',', '.')).astype(float)
#Se existir casos de quantidade de item = 0, remove
df_u = df_u.loc[df_u['QUANTIDADE_ITEM'] > 0]
df_u['VALOR_ITEM_UNIT'] = df_u['VALOR_ITEM']/df_u['QUANTIDADE_ITEM']
# Retira coluna quantidade e valor de item porque determinam valor licitação
del df_u['QUANTIDADE_ITEM']
del df_u['VALOR_ITEM']

In [ ]:
n = gc.collect()
print("Número de objetos coletados pela função GC:", n)
print("Objetos restantes:", gc.garbage)

In [ ]:
df_u['DESCRICAO_ITEM_COMPRA'] = df_u[['DESCRICAO_ITEM_COMPRA']].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))

In [ ]:
n = gc.collect()
print("Número de objetos coletados pela função GC:", n)
print("Objetos restantes:", gc.garbage)

In [ ]:
df_u['NOME_PARTICIPANTE'] = df_u[['NOME_PARTICIPANTE']].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))

In [ ]:
n = gc.collect()
print("Número de objetos coletados pela função GC:", n)
print("Objetos restantes:", gc.garbage)

In [ ]:
df_u['NOME_VENCEDOR'] = df_u[['NOME_VENCEDOR']].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))

In [ ]:
df_u = df_u.drop_duplicates()

In [ ]:
print (df_u.shape)
print (df_u.columns)

In [ ]:
#backup
df_u.to_csv('C:\\Users\\lucas\\Downloads\\Cefet-RJ\\licitacoes_tratadas_bkp.csv', sep=';', index=False)

In [ ]:
#Carregar dados, quando os passos anteriores já tiverem sido executados
df_u = pd.read_csv('C:\\Users\\lucas\\Downloads\\Cefet-RJ\\licitacoes_tratadas_bkp.csv', sep=';', encoding='utf-8', low_memory=True)
df_u['RESULTADO_COMPRA_ANO'] = df_u['RESULTADO_COMPRA_ANO'].astype(str)

### Acrescenta informações sobre conluio dados de conluios

In [ ]:
#Acrescenta classificação de conluio 
for index, row in df_cc.iterrows():
    df_u.loc[(df_u['NOME_PARTICIPANTE'] == row['NOME_PARTICIPANTE']) & (df_u['RESULTADO_COMPRA_ANO'].isin(str(row['PERIODO']).split(', '))), 'PARTICIPA_CONLUIO_PERIODO'] = 'SIM'

In [ ]:
df_u_count = df_u.loc[df_u['PARTICIPA_CONLUIO_PERIODO'] == 'SIM']

In [ ]:
df_u_count = df_u_count[['NUMERO_LICITACAO', 'NOME_PARTICIPANTE']].drop_duplicates()

In [ ]:
df_u_count['QTD_PARTICIPANTES_CONLUIO'] = df_u_count.groupby(by=['NUMERO_LICITACAO'])['NOME_PARTICIPANTE'].transform('count')

In [ ]:
df_u_count.head()

In [ ]:
df_u = df_u.merge(df_u_count, how='left')

In [ ]:
df_u.loc[df_u['PARTICIPA_CONLUIO_PERIODO'] == 'SIM'].head()

In [ ]:
df_u.loc[(df_u['QTD_PARTICIPANTES_CONLUIO']>1) & (df_u['PARTICIPA_CONLUIO_PERIODO'] == 'SIM'), 'LICITACAO_CONLUIO'] = 'SIM'

In [ ]:
df_u.loc[df_u['LICITACAO_CONLUIO']== 'SIM'].head()

In [ ]:
del(df_u_count)
# Baixa variabilidade. Exclui após utilização para sumarização.
df_u.drop(columns='NUMERO_LICITACAO', axis=1, inplace=True)

In [ ]:
df_u.loc[df_u['LICITACAO_CONLUIO'].isna(), 'LICITACAO_CONLUIO'] = 'NAO'
df_u.loc[df_u['PARTICIPA_CONLUIO_PERIODO'].isna(), 'PARTICIPA_CONLUIO_PERIODO'] = 'NAO'
df_u.loc[df_u['QTD_PARTICIPANTES_CONLUIO'].isna(), 'QTD_PARTICIPANTES_CONLUIO'] = 0
#Elimina atributos utilizados para calculo da classificao, para não enviesar a determinação
df_u = df_u.drop(columns=['PARTICIPA_CONLUIO_PERIODO', 'QTD_PARTICIPANTES_CONLUIO'], axis=1)

In [ ]:
print (df_u.shape)
print (df_u.columns)

In [ ]:
print (df_u.loc[df_u['LICITACAO_CONLUIO']== 'SIM'].shape)
print (df_u.loc[df_u['LICITACAO_CONLUIO']== 'NAO'].shape)

## Enriquecimento do conjunto de dados

### Acrescenta variáveis estatística

In [ ]:
import statistics as stat
import scipy.stats as stats

In [ ]:
#Calcular estatisticas do valor unitario do item
df_u['MEDIA_ITEM'] = df_u.groupby('DESCRICAO_ITEM_COMPRA')['VALOR_ITEM_UNIT'].transform('mean')
#df_u['DESVIO_PADRAO_ITEM'] = df_u.groupby(['DESCRICAO_ITEM_COMPRA'])['VALOR_ITEM_UNIT'].transform(lambda x: stat.stdev(x) if len(x) > 1 else 0)
df_u['COEFICIENTE_VARIACAO_ITEM'] = df_u.groupby(['DESCRICAO_ITEM_COMPRA'])['VALOR_ITEM_UNIT'].transform(lambda x: stat.stdev(x)/x.mean() if len(x) > 1 else 0)
df_u['CURTOSE_ITEM'] = df_u.groupby(['DESCRICAO_ITEM_COMPRA'])['VALOR_ITEM_UNIT'].transform(lambda x: stats.kurtosis(x) if len(x) > 1 else 0)
df_u['ASSIMETRIA'] = df_u.groupby(['DESCRICAO_ITEM_COMPRA'])['VALOR_ITEM_UNIT'].transform(lambda x: stats.skew(x) if len(x) > 1 else 0)
df_u['3_QUARTIL_ITEM'] = df_u.groupby(['DESCRICAO_ITEM_COMPRA'])['VALOR_ITEM_UNIT'].transform(lambda x: x.quantile(0.75) if len(x) > 1 else 0)
df_u['OUTLIER'] = df_u.apply(lambda x: 'SIM' if x['VALOR_ITEM_UNIT'] > x['3_QUARTIL_ITEM'] else 'NAO', axis=1)

In [ ]:
#Exclui atributos utilizados para calcular outros
df_u = df_u.drop(columns=['MEDIA_ITEM', '3_QUARTIL_ITEM', 'VALOR_ITEM_UNIT'], axis=1)#, 'DESVIO_PADRAO_ITEM'

In [ ]:
df_u.head()

In [ ]:
n = gc.collect()
print("Número de objetos coletados pela função GC:", n)
print("Objetos restantes:", gc.garbage)

In [ ]:
df_u.to_csv('C:\\Users\\lucas\\Downloads\\Cefet-RJ\\licitacoes_conluios_tratadas.csv', sep=';', index=False)

In [5]:
#Carregar dados, quando os passos anteriores já tiverem sido executados
df_u = pd.read_csv('C:\\Users\\lucas\\Downloads\\Cefet-RJ\\licitacoes_conluios_tratadas.csv', sep=';', encoding='utf-8', low_memory=True)
df_u['RESULTADO_COMPRA_ANO'] = df_u['RESULTADO_COMPRA_ANO'].astype(str)

## Pré-processamento - parte 2

In [6]:
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error
from sklearn.metrics import matthews_corrcoef

In [7]:
def executar_modelo(name, estat, clf, X_train2, X_test2):
    print("Classificação do modelo: Linear SVM", name, ". Variáveis estatísticas: ",  estat)
    clf.fit(X_train2, y_train)
    print("Acurácia: {:.3f}".format(clf.score(X_test2, y_test)))
    y_pred = clf.predict(X_test2)
    print("Acurácia: {:.3f}".format(metrics.accuracy_score(y_test, y_pred)))

    #Métricas gerais
    mse = mean_squared_error(y_test, y_pred)
    print('Mean Squared Error (MSE):', mse)
    mcc = matthews_corrcoef(y_test, y_pred)
    print('Matthews correlation coefficient (MCC):', mcc)
    print(classification_report(y_pred, y_test))

    return clf

In [8]:
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay
def gera_curva_roc(name, clf_ce, clf_se):
    _, ax = plt.subplots() 
    plt.plot([0, 1], [0, 1], 'k--', label='Classificador aleatório')
    RocCurveDisplay.from_estimator(clf_ce, X_test, y_test, ax=ax, name=name+" (COM var. estat.)")
    RocCurveDisplay.from_estimator(clf_se, X_test[:,0:5], y_test, ax=ax, name=name+" (SEM var. estat.)")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Taxa de Positivos Falsos')
    plt.ylabel('Taxa de Positivos Verdadeiros')
    plt.legend()
    plt.show()

In [9]:
from sklearn.model_selection import KFold, cross_validate
def executa_validacao_cruzada(clf, X, y):
    k_folds = KFold(n_splits = 10)
    scoring = ['accuracy', 'precision', 'recall']
    scores = cross_validate(clf, X, y, cv = k_folds, scoring = scoring)
    print("Cross Validation Scores - Média")
    print("Acurácia: ", scores['test_accuracy'].mean())
    print("Precisão: ", scores['test_precision'].mean())
    print("Recall: ", scores['test_recall'].mean())

In [10]:
import shap
def explicar_modelo_shap (name, clf):
    print('Explicação do modelo com SHAP')
    shap.initjs()
    X_train_summary = shap.kmeans(X_train, 3)
    explainer = shap.KernelExplainer(clf.predict, X_train_summary)   
    #shap_values = explainer.shap_values(X_test)
    shap_values = explainer.shap_values(X_test, nsamples=1000)
    figure = plt.figure()
    shap.summary_plot(shap_values, X_test)

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [11]:
from lime.lime_tabular import LimeTabularExplainer
def explicar_modelo_lime (clf, i):
    print('Explicação do modelo com LIME')
    explainer = LimeTabularExplainer(X_train, feature_names=X_columns, class_names=['Flag Vencedor'], verbose=True, mode='regression')
    exp = explainer.explain_instance(X_test[i], model.predict, num_features=5)
    print('Instância:', i)
    print('Predição:', y_pred[i])
    exp.show_in_notebook(show_table=True)

### Codificação e balanceamento

In [12]:
df_u_scaled = escalar_df(df_u)

In [13]:
df_u_scaled = df_u_scaled.reindex(columns=["NOME_UG", "NOME_PARTICIPANTE", "NOME_VENCEDOR", "MODALIDADE_COMPRA", "DESCRICAO_ITEM_COMPRA", "SITUACAO_LICITACAO", "UF", "RESULTADO_COMPRA_MES", "RESULTADO_COMPRA_ANO", "LICITACAO_CONLUIO", "QTD_PARTICIPANTES_ITEM", "OUTLIER", "COEFICIENTE_VARIACAO_ITEM", "CURTOSE_ITEM", "ASSIMETRIA"])

In [14]:
df_u_scaled.iloc[:,0:10].head()

,NOME_UG,NOME_PARTICIPANTE,NOME_VENCEDOR,MODALIDADE_COMPRA,DESCRICAO_ITEM_COMPRA,SITUACAO_LICITACAO,UF,RESULTADO_COMPRA_MES,RESULTADO_COMPRA_ANO,LICITACAO_CONLUIO
0,0.136912,0.836786,0.256697,1.0,0.268126,1.0,0.703704,0.0,0.0,0.0
1,0.136912,0.019252,0.256697,1.0,0.268126,1.0,0.703704,0.0,0.0,0.0
2,0.136912,0.093112,0.256697,1.0,0.268126,1.0,0.703704,0.0,0.0,0.0
3,0.136912,0.392152,0.256697,1.0,0.268126,1.0,0.703704,0.0,0.0,0.0
4,0.136912,0.811847,0.256697,1.0,0.268126,1.0,0.703704,0.0,0.0,0.0


In [15]:
df_u_scaledX = df_u_scaled.drop(columns=['LICITACAO_CONLUIO'], axis=1)
df_u_scaledY = df_u_scaled['LICITACAO_CONLUIO']

In [16]:
X = df_u_scaledX.values
X_columns = df_u_scaledX.columns
y = df_u_scaledY.values
y_columns = ['LICITACAO_CONLUIO']

In [17]:
del df_u
del df_u_scaled
del df_u_scaledX
del df_u_scaledY
gc.collect()

0

In [18]:
#Definir a estratégia de balanceamento
smode = 'smote'
if (smode == 'under'):
    from imblearn.under_sampling import RandomUnderSampler
    samp = RandomUnderSampler(random_state=42, sampling_strategy = 'majority')
elif (smode == 'over'):
    from imblearn.over_sampling import RandomOverSampler
    samp = RandomOverSampler(sampling_strategy='minority')
elif (smode == 'smote'):
    from imblearn.over_sampling import SMOTE
    samp = SMOTE()
elif (smode == 'smoteenn'):
    from imblearn.combine import SMOTEENN
    samp = SMOTEENN(random_state=42)  
if (smode != 'na'):
    X, y = samp.fit_resample(X, y)
print ("Balanceamento selecionado: "+smode+" - Quantidade 0: "+str((y==0).sum())+" - Quantidade 1: "+str((y==1).sum()))

Balanceamento selecionado: smote - Quantidade 0: 33914703 - Quantidade 1: 33914703


In [ ]:
#Avalia métricas para balanceamento
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print ("Quantidade de teste 0: "+str((X_test==0).sum())+" - Quantidade de teste 1: "+str((y_test==1).sum()))
from sklearn.svm import LinearSVC
print ("Balanceamento: ", smode)
executar_modelo("Linear SVM", "SIM", LinearSVC(), X_train, X_test)
executar_modelo("Linear SVM", "NÃO", LinearSVC(), X_train[:,0:5], X_test[:,0:5])

Quantidade de teste 0: 19164847 - Quantidade de teste 1: 6782941
Balanceamento:  smote
Classificação do modelo: Linear SVM Linear SVM . Variáveis estatísticas:  SIM


### Seleção de atributos

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif, RFE
from sklearn.tree import DecisionTreeClassifier

In [ ]:
#Imprimi gráfico - Chi2
fs = SelectKBest(score_func=chi2, k='all')
fs.fit(X[:,0:9], y)
plt.bar([i for i in range(len(fs.scores_))], fs.scores_)
plt.show()

In [ ]:
#Imprimi gráfico - Seleção univariada
fs = SelectKBest(score_func=f_classif, k=5)
fs.fit(X[:,0:9], y)
plt.bar([i for i in range(len(fs.scores_))], fs.scores_)
plt.show()

In [ ]:
#Imprimi gráfico - Mutual information
fs = SelectKBest(score_func=mutual_info_classif, k=5)
fs.fit(X[:,0:9], y)
plt.bar([i for i in range(len(fs.scores_))], fs.scores_)
plt.show()

In [ ]:
#Imprimi ranking - RFE
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=5)
rfe.fit(X[:,0:9], y)
for i in range(X[:,0:9].shape[1]):
    print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))

In [ ]:
#Imprimi gráfico - Feature importance
#feature_names = [f"{i}" for i in range(X.shape[1])]
#model = ExtraTreesClassifier()
#model.fit(X[:,0:9], y)
#std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)
#forest_importances = pd.Series(model.feature_importances_, index=feature_names)
#forest_importances.plot.bar(yerr=std)

In [ ]:
#Definir a estratégia de seleção de atributos
smode = 'Chi2'
X2 = X
if (smode == 'Chi2'): #Chi2
    selector = SelectKBest(chi2, k=5)
    X2 = selector.fit_transform(X[:,0:9], y)
elif (smode == 'SU'): #Seleção univariada
    selector = SelectKBest(f_classif, k=5)
    X2 = selector.fit_transform(X[:,0:9], y)
elif (smode == 'MI'): #Mutual information
    selector = SelectKBest(mutual_info_classif, k=5)
    X2 = selector.fit_transform(X[:,0:9], y)    
    mutual_info_classif
elif (smode == 'RFE'): #Recursive Feature Elimination
    selector = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=5)
    X2 = selector.fit_transform(X[:,0:9], y)
elif (smode == 'PCA'): #PCA
    from sklearn.decomposition import PCA
    selector = PCA(n_components=5)
    X2 = selector.fit_transform(X[:,0:9], y)
#elif (smode == 'ETC'):#Feature importance com Extra Trees Classifier
#    from sklearn.decomposition import PCA
#    selector = ExtraTreesClassifier()
#    model.fit(X[:,0:9], y)
#    std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)

In [ ]:
#Nome das colunas selecionadas
X2_columns = X_columns[0:9]
print(X2_columns[selector.get_support()])

In [ ]:
import numpy as np
X = np.concatenate((X2, X[:,9:]), axis=1)

In [ ]:
del X2
del X2_columns
gc.collect()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print ("Quantidade de teste 0: "+str((X_test==0).sum())+" - Quantidade de teste 1: "+str((y_test==1).sum()))

In [ ]:
#Avalia métricas feature selecion
from sklearn.svm import LinearSVC
print ("Feature selecion: ", smode)
executar_modelo("Linear SVM", "SIM", LinearSVC(), X_train, X_test)
executar_modelo("Linear SVM", "NÃO", LinearSVC(), X_train[:,0:5], X_test[:,0:5])

## Aprendizado

In [ ]:
n = gc.collect()
print("Número de objetos coletados pela função GC:", n)
print("Objetos restantes:", gc.garbage)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf_ce = executar_modelo("Árvore de decisão", "SIM", DecisionTreeClassifier(max_depth=5), X_train, X_test)
executa_validacao_cruzada(clf_ce, X, y)
clf_se = executar_modelo("Árvore de decisão", "NÃO", DecisionTreeClassifier(max_depth=5), X_train[:,0:5], X_test[:,0:5])
executa_validacao_cruzada(clf_se, X[:,0:5], y)
gera_curva_roc("Árvore de decisão", clf_ce, clf_se)
explicar_modelo_shap ("Árvore de decisão", clf_ce)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf_ce = executar_modelo("Regressão Logística", "SIM", LogisticRegression(), X_train, X_test)
executa_validacao_cruzada(clf_ce, X, y)
clf_se = executar_modelo("Regressão Logística", "NÃO", LogisticRegression(), X_train[:,0:5], X_test[:,0:5])
executa_validacao_cruzada(clf_se, X[:,0:5], y)
gera_curva_roc("Regressão Logística", clf_ce, clf_se)
explicar_modelo_shap ("Regressão Logística", clf_ce)

In [ ]:
from sklearn.naive_bayes import GaussianNB
clf_ce = executar_modelo("Naive Bayes", "SIM", GaussianNB(), X_train, X_test)
executa_validacao_cruzada(clf_ce, X, y)
clf_se = executar_modelo("Naive Bayes", "NÃO", GaussianNB(), X_train[:,0:5], X_test[:,0:5])
executa_validacao_cruzada(clf_se, X[:,0:5], y)
gera_curva_roc("Naive Bayes", clf_ce, clf_se)
explicar_modelo_shap ("Naive Bayes", clf_ce)

In [ ]:
from sklearn.svm import LinearSVC
clf_ce = executar_modelo("Linear SVM", "SIM", LinearSVC(), X_train, X_test)
executa_validacao_cruzada(clf_ce, X, y)
clf_se = executar_modelo("Linear SVM", "NÃO", LinearSVC(), X_train[:,0:5], X_test[:,0:5])
executa_validacao_cruzada(clf_se, X[:,0:5], y)
gera_curva_roc("Linear SVM", clf_ce, clf_se)
explicar_modelo_shap ("Linear SVM", clf_ce)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf_ce = executar_modelo("Random Forest", "SIM", RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1), X_train, X_test)
executa_validacao_cruzada(clf_ce, X, y)
clf_se = executar_modelo("Random Forest", "NÃO", RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1), X_train[:,0:5], X_test[:,0:5])
executa_validacao_cruzada(clf_se, X[:,0:5], y)
gera_curva_roc("Random Forest", clf_ce, clf_se)
explicar_modelo_shap ("Random Forest", clf_ce)

In [ ]:
from sklearn.neural_network import MLPClassifier
clf_ce = executar_modelo("MLP", "SIM", MLPClassifier(), X_train, X_test)
executa_validacao_cruzada(clf_ce, X, y)
clf_se = executar_modelo("MLP", "NÃO", MLPClassifier(), X_train[:,0:5], X_test[:,0:5])
executa_validacao_cruzada(clf_se, X[:,0:5], y)
gera_curva_roc("MLP", clf_ce, clf_se)
explicar_modelo_shap ("Neural Net", clf_ce)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
clf_ce = executar_modelo("AdaBoost", "SIM", AdaBoostClassifier(), X_train, X_test)
executa_validacao_cruzada(clf_ce, X, y)
clf_se = executar_modelo("AdaBoost", "NÃO", AdaBoostClassifier(), X_train[:,0:5], X_test[:,0:5])
executa_validacao_cruzada(clf_se, X[:,0:5], y)
gera_curva_roc("AdaBoost", clf_ce, clf_se)
explicar_modelo_shap ("AdaBoost", clf_ce)

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
clf_ce = executar_modelo("QDA", "SIM", QuadraticDiscriminantAnalysis(), X_train, X_test)
executa_validacao_cruzada(clf_ce, X, y)
clf_se = executar_modelo("QDA", "NÃO", QuadraticDiscriminantAnalysis(), X_train[:,0:5], X_test[:,0:5])
executa_validacao_cruzada(clf_se, X[:,0:5], y)
gera_curva_roc("QDA", clf_ce, clf_se)
explicar_modelo_shap ("Quadratic Discriminant Analysis", clf_ce)

In [ ]:
from sklearn.linear_model import RidgeClassifierCV
clf_ce = executar_modelo("Ridge", "SIM", RidgeClassifierCV(), X_train, X_test)
executa_validacao_cruzada(clf_ce, X, y)
clf_se = executar_modelo("Ridge", "NÃO", RidgeClassifierCV(), X_train[:,0:5], X_test[:,0:5])
executa_validacao_cruzada(clf_se, X[:,0:5], y)
gera_curva_roc("Ridge", clf_ce, clf_se)
explicar_modelo_shap ("Ridge", clf_ce)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf_ce = executar_modelo("K-NN", "SIM", KNeighborsClassifier(n_neighbors=5, n_jobs=-1), X_train, X_test)
executa_validacao_cruzada(clf_ce, X, y)
clf_se = executar_modelo("K-NN", "NÃO", KNeighborsClassifier(n_neighbors=5, n_jobs=-1), X_train[:,0:5], X_test[:,0:5])
executa_validacao_cruzada(clf_se, X[:,0:5], y)
gera_curva_roc("K-NN", clf_ce, clf_se)
explicar_modelo_shap ("Nearest Neighbors", clf_ce)